<a href="https://colab.research.google.com/github/Nisha789/Apache-Spark/blob/main/Joins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e298032d86c91ce86cf4eca8d851e4abf9a3b4ef37d1beb36962e852809da250
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
        .appName("Spark_Colab")\
        .getOrCreate()

In [ ]:
spark

In [ ]:
person = spark.createDataFrame([
    (0,"Bill Chambers",0,[100]),
    (1,"Matei Zaharia",1,[500,250,100]),
    (2,"Michael Armbrust",3,[250,100])
]).toDF("id","name","graduate_program","spark_status")

In [ ]:
person.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+



In [ ]:
person = spark.createDataFrame([
  (0, "Bill Chambers", 0, [100]),
  (1, "Matei Zaharia", 1, [500, 250, 100]),
  (2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")

graduateProgram = spark.createDataFrame([
  (0, "Masters", "School of Information", "UC Berkeley"),
  (2, "Masters", "EECS", "UC Berkeley"),
  (1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")

sparkStatus = spark.createDataFrame([
  (500, "Vice President"),
  (250, "PMC Member"),
  (100, "Contributor")])\
.toDF("id", "status")

In [ ]:
person.show()
graduateProgram.show()
sparkStatus.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               3|     [250, 100]|
+---+----------------+----------------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



In [ ]:
joinExpression = person["graduate_program"] == graduateProgram["id"]

In [ ]:
# Inner Join - Default
from pyspark.sql.functions import col
person.join(graduateProgram,joinExpression).show()
# .select(person["id"],"name","degree",graduateProgram["id"])

+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
| id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|  1|Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------------+----------------+---------------+---+-------+--------------------+-----------+



In [ ]:
# Outer Join - Evaluate expr - true or false - ok - for false, it put NULL
joinType = "outer"
person.join(graduateProgram,joinExpression,joinType).show()

+----+----------------+----------------+---------------+----+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status|  id| degree|          department|     school|
+----+----------------+----------------+---------------+----+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|   0|Masters|School of Informa...|UC Berkeley|
|   1|   Matei Zaharia|               1|[500, 250, 100]|   1|  Ph.D.|                EECS|UC Berkeley|
|NULL|            NULL|            NULL|           NULL|   2|Masters|                EECS|UC Berkeley|
|   2|Michael Armbrust|               3|     [250, 100]|NULL|   NULL|                NULL|       NULL|
+----+----------------+----------------+---------------+----+-------+--------------------+-----------+



In [ ]:
joinType = "left_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+---+----------------+----------------+---------------+----+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|  id| degree|          department|     school|
+---+----------------+----------------+---------------+----+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|   0|Masters|School of Informa...|UC Berkeley|
|  1|   Matei Zaharia|               1|[500, 250, 100]|   1|  Ph.D.|                EECS|UC Berkeley|
|  2|Michael Armbrust|               3|     [250, 100]|NULL|   NULL|                NULL|       NULL|
+---+----------------+----------------+---------------+----+-------+--------------------+-----------+



In [ ]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|         name|graduate_program|   spark_status| id| degree|          department|     school|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkeley|
|   1|Matei Zaharia|               1|[500, 250, 100]|  1|  Ph.D.|                EECS|UC Berkeley|
|NULL|         NULL|            NULL|           NULL|  2|Masters|                EECS|UC Berkeley|
+----+-------------+----------------+---------------+---+-------+--------------------+-----------+



In [ ]:
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+



In [ ]:
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+-----------+
| id| degree|department|     school|
+---+-------+----------+-----------+
|  2|Masters|      EECS|UC Berkeley|
+---+-------+----------+-----------+



In [ ]:
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkeley|  0|   Bill Chambers|               0|          [100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  1|  Ph.D.|                EECS|UC Berkeley|  2|Michael Armbrust|               1|     [250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



In [ ]:
person.join(graduateProgram,joinExpression).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#1705L], [id#1719L], Inner
   :- Sort [graduate_program#1705L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#1705L, 200), ENSURE_REQUIREMENTS, [plan_id=3561]
   :     +- Project [_1#1695L AS id#1703L, _2#1696 AS name#1704, _3#1697L AS graduate_program#1705L, _4#1698 AS spark_status#1706]
   :        +- Filter isnotnull(_3#1697L)
   :           +- Scan ExistingRDD[_1#1695L,_2#1696,_3#1697L,_4#1698]
   +- Sort [id#1719L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#1719L, 200), ENSURE_REQUIREMENTS, [plan_id=3562]
         +- Project [_1#1711L AS id#1719L, _2#1712 AS degree#1720, _3#1713 AS department#1721, _4#1714 AS school#1722]
            +- Filter isnotnull(_1#1711L)
               +- Scan ExistingRDD[_1#1711L,_2#1712,_3#1713,_4#1714]


